In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble  import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
import random

from sklearn import metrics
from collections import Counter
import argparse
from sklearn.model_selection import cross_validate
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score, precision_score, recall_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import ParameterGrid
import lightgbm as lgb
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, LabelBinarizer

import collections

In [2]:
def other(x):
    if ((x=='exchanges')|(x=='finance')|(x=='games')|(x=='gambling')|(x=='high-risk')): 
        return x 
    else: 
        return 'other'

In [3]:
df = pd.read_csv('drive/MyDrive/Colab Notebooks/IRTG/Smart_contracts_paper/USC/SC-Dapp-scraping/data/dapps_source_code_parsed_open_source_2021-01-08.csv')
df = df[df.SC_full!='\n']
df.drop('Unnamed: 0', axis=1, inplace=True)
df.category = df.category.apply(other)
df.comments.fillna('empty', inplace=True)

unlabelled = pd.read_csv('drive/MyDrive/Colab Notebooks/IRTG/Smart_contracts_paper/USC/SC-open-source-source-code-scraping/data/open_source_parsed_16thousand.csv')
unlabelled = unlabelled[unlabelled.SC_full!='\n']
unlabelled.comments.fillna('empty', inplace=True)

In [4]:
X = df.drop('category', axis=1)
y = df.category
X_test = unlabelled
RS     = 42

## WITHOUT Comments, only code

### Ridge Regression

In [5]:
c = 0
C_parameter = np.arange(0.1, 1.1, 0.1) 
FEATURES    = np.arange(500, 7700, 200)

# use best C

C_parameter = [1]
FEATURES    = [7300]

In [6]:
# TRAIN ON WHOLE DATA AND PREDICT ON TEST
probs_final = pd.DataFrame()
word_vectorizer   = TfidfVectorizer(
                    sublinear_tf  = True,
                    strip_accents = 'unicode',
                    analyzer      = 'word',
                    token_pattern = r'\w{1,}',
                    ngram_range   = (1, 1),
                    max_features  = FEATURES[0])
word_vectorizer.fit(X['SC_no_comments'])
train_word_features  = word_vectorizer.transform(X['SC_no_comments'])
test_word_features   = word_vectorizer.transform(X_test['SC_no_comments'])

le = LabelBinarizer()
le.fit(y)

y_train = le.transform(y)

for i, cl in enumerate(le.classes_):
    classifier    = LogisticRegression(C=C_parameter[0], 
                                    solver='sag', 
                                    random_state=RS)
    classifier.fit(train_word_features, y_train[:,i])
    probs           = classifier.predict_proba(test_word_features)[:,1]
    probs_final[cl] = probs

In [7]:
probs_final.idxmax(axis=1).value_counts()

finance      10009
other         4757
exchanges      756
games          506
high-risk      162
gambling        60
dtype: int64

In [10]:
final

,hash,SC_no_comments,exchanges,finance,gambling,games,high-risk,other,class
0,0xd8b5f514b97a53edc9107111e59a2d18424d9574,pragma solidity ^0.4.26;\n\nlibrary SafeMath {...,0.105392,0.200651,0.044863,0.122575,0.024257,0.421082,other
1,0x0be2ea1062e62632edb47772b8970ada04acd387,pragma solidity ^0.4.24;\n\n\n\ncontract UtilF...,0.088841,0.112755,0.099562,0.173366,0.090025,0.236691,other
2,0x02e75fe4e8f8531421ee22f4560664685c3f07de,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,0.162814,0.509107,0.051351,0.064330,0.043472,0.075876,finance
3,0x9ff2e4f332d91ccc3ab368657124d6b736b64590,\n\n\n\npragma solidity 0.5.12;\n\n\n\n\n\nlib...,0.091531,0.392411,0.075271,0.108037,0.027437,0.190886,finance
4,0x705e97b137f3b38e8d151b5619b436e2d8a2882a,pragma solidity >= 0.5.11;\n\n\n\n\n\ninterfac...,0.077116,0.232719,0.073430,0.132689,0.070177,0.254400,other
...,...,...,...,...,...,...,...,...,...
16245,0x3aaFb0E5b57bb19D02FcB6656059B34d8E79471f,pragma solidity 0.6.6;\n\n\n\nlibrary SafeMath...,0.102381,0.356989,0.080634,0.104295,0.041418,0.165756,finance
16246,0x68914f24aa57e7269f587a105fc2a9ebb994ab8f,pragma solidity ^0.5.12;\n\n\n\n\n\ncontract T...,0.195563,0.241771,0.026205,0.081734,0.025867,0.282097,other
16247,0xb260687cc277c0BFF48c9C482c16BC62f4eBa927,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,0.220024,0.232312,0.052658,0.091027,0.033643,0.137453,finance
16248,0x1149cfc37d81304173cd2b203508ab1f6934b81b,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,0.177954,0.284850,0.037934,0.135217,0.023136,0.282193,finance


In [9]:
final = X_test[['hash', 'SC_no_comments']]
final[list(probs_final.columns)] = probs_final
final['class'] = probs_final.idxmax(axis=1)
final.to_csv('drive/MyDrive/Colab Notebooks/IRTG/Smart_contracts_paper/USC/SC-classification/output_data/final_probs_unlabelled.csv', index=False)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
